In [4]:
# This kernel have improvement from Pranav Pandya and Andy Harless
# Pranav Kernel: https://www.kaggle.com/pranav84/xgboost-on-hist-mode-ip-addresses-dropped
# Andy Kernel: https://www.kaggle.com/aharless/jo-o-s-xgboost-with-memory-usage-enhancements

import gc
import time
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split
import xgboost as xgb
from xgboost import plot_importance
import matplotlib.pyplot as plt

# Change this for validation with 10% from train
is_valid = False

path = './input/train.csv'

train_df = pd.read_csv(path)

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [21]:
# This kernel have improvement from Pranav Pandya and Andy Harless
# Pranav Kernel: https://www.kaggle.com/pranav84/xgboost-on-hist-mode-ip-addresses-dropped
# Andy Kernel: https://www.kaggle.com/aharless/jo-o-s-xgboost-with-memory-usage-enhancements

import gc
import time
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split
import xgboost as xgb
from xgboost import plot_importance
import matplotlib.pyplot as plt

# Change this for validation with 10% from train
is_valid = False

path = 'train_sample.csv'

def timeFeatures(df):
    # Make some new features with click_time column
    df['datetime'] = pd.to_datetime(df['click_time'])
    df['dow']      = df['datetime'].dt.dayofweek
    df["doy"]      = df["datetime"].dt.dayofyear
    #df["dteom"]    = df["datetime"].dt.daysinmonth - df["datetime"].dt.day
    df.drop(['click_time', 'datetime'], axis=1, inplace=True)
    return df

start_time = time.time()

train_columns = ['ip', 'app', 'device', 'os', 'channel', 'click_time', 'is_attributed']
test_columns  = ['ip', 'app', 'device', 'os', 'channel', 'click_time', 'click_id']
dtypes = {
        'ip'            : 'uint32',
        'app'           : 'uint16',
        'device'        : 'uint16',
        'os'            : 'uint16',
        'channel'       : 'uint16',
        'is_attributed' : 'uint8',
        'click_id'      : 'uint32'
        }

# Read the last lines because they are more impacting in training than the starting lines
train_df = pd.read_csv(path)
print('train_df.columns.values\n', train_df.columns.values)
print(train_df.shape)

ip_count = merge.groupby(['ip'])['channel'].count().reset_index()
print('ip_count\n', ip_count)

train_df.columns.values
 ['ip' 'app' 'device' 'os' 'channel' 'click_time' 'attributed_time'
 'is_attributed']
(100000, 8)
ip_count
             ip  channel
0            0        2
1            1       50
2            2        6
3            3      291
4            4       39
5            5       99
6            6      798
7            7        1
8            8        2
9            9     1385
10          10      598
11          11        9
12          12       30
13          13       27
14          14        4
15          15        1
16          16       13
17          17        1
18          18       18
19          19      415
20          20     1827
21          21       38
22          22       15
23          23        4
24          24        4
25          25      608
26          26       21
27          27      910
28          28        3
29          29       93
...        ...      ...
126384  126384       37
126385  126385        1
126386  126386     1214
126387  126387        2
1263

In [23]:
# This kernel have improvement from Pranav Pandya and Andy Harless
# Pranav Kernel: https://www.kaggle.com/pranav84/xgboost-on-hist-mode-ip-addresses-dropped
# Andy Kernel: https://www.kaggle.com/aharless/jo-o-s-xgboost-with-memory-usage-enhancements

import gc
import time
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split
import xgboost as xgb
from xgboost import plot_importance
import matplotlib.pyplot as plt

# Change this for validation with 10% from train
is_valid = False

path_train = './input/train_sample.csv'
path_test = './input/test_supplement.csv'

def timeFeatures(df):
    # Make some new features with click_time column
    df['datetime'] = pd.to_datetime(df['click_time'])
    df['dow']      = df['datetime'].dt.dayofweek
    df["doy"]      = df["datetime"].dt.dayofyear
    #df["dteom"]    = df["datetime"].dt.daysinmonth - df["datetime"].dt.day
    df.drop(['click_time', 'datetime'], axis=1, inplace=True)
    return df

start_time = time.time()

train_columns = ['ip', 'app', 'device', 'os', 'channel', 'click_time', 'is_attributed']
test_columns  = ['ip', 'app', 'device', 'os', 'channel', 'click_time', 'click_id']
dtypes = {
        'ip'            : 'uint32',
        'app'           : 'uint16',
        'device'        : 'uint16',
        'os'            : 'uint16',
        'channel'       : 'uint16',
        'is_attributed' : 'uint8',
        'click_id'      : 'uint32'
        }

# Read the last lines because they are more impacting in training than the starting lines
train_df = pd.read_csv(path_train)
print('train_df.columns.values\n',train_df.columns.values)
# params: usecols-какие столбцы в данных будем использовать, skiprows-пропуст строк в диапозоне, nrows-чтение кол-во строк,   
train_df = pd.read_csv(path_train, skiprows=range(1,123903891), nrows=61000000, usecols=train_columns, dtype=dtypes)
test_df = pd.read_csv(path_test, usecols=test_columns, dtype=dtypes)

print('[{}] Finished to load data'.format(time.time() - start_time))

# Drop the IP and the columns from target
y = train_df['is_attributed']
#  params: axis-whether to drop labels from the index (0/‘index’) or columns (1/‘columns’),
# inplace-If True, do operation inplace and return None.
train_df.drop(['is_attributed'], axis=1, inplace=True)

# Drop IP and ID from test rows
sub = pd.DataFrame()
#sub['click_id'] = test['click_id'].astype('int')
test_df.drop(['click_id'], axis=1, inplace=True)
# сборщик мусора
gc.collect()
# кол-во строк
nrow_train = train_df.shape[0]
# объединяем две выборки
merge = pd.concat([train_df, test_df])

del train_df, test_df
gc.collect()

# Count the number of clicks by ip
ip_count = merge.groupby(['ip'])['channel'].count().reset_index()
# print('ip_count', ip_count)
ip_count.columns = ['ip', 'clicks_by_ip']
# params: merge-выборка, включающая train_df, test_df. on-имена полей для присоединения, 
# how-использует только ключи из левого кадра, похожие на внешнее соединение SQL слева
merge = pd.merge(merge, ip_count, on='ip', how='left', sort=False)
merge['clicks_by_ip'] = merge['clicks_by_ip'].astype('uint16')
merge.drop('ip', axis=1, inplace=True)

train_df = merge[:nrow_train]
test_df = merge[nrow_train:]

del test_df, merge
gc.collect()

print('[{}] Start to generate time features'.format(time.time() - start_time))

train_df = timeFeatures(train_df)
gc.collect()

print('[{}] Start XGBoost Training'.format(time.time() - start_time))

# Set the params(this params from Pranav kernel) for xgboost model
params = {'eta': 0.3,
          'tree_method': "hist",
          'grow_policy': "lossguide",
          'max_leaves': 1400,  
          'max_depth': 0, 
          'subsample': 0.9, 
          'colsample_bytree': 0.7, 
          'colsample_bylevel':0.7,
          'min_child_weight':0,
          'alpha':4,
          'objective': 'binary:logistic', 
          'scale_pos_weight':9,
          'eval_metric': 'auc', 
          'nthread':8,
          'random_state': 99, 
          'silent': True}
          

if (is_valid == True):
    # Get 10% of train dataset to use as validation
    x1, x2, y1, y2 = train_test_split(train_df, y, test_size=0.1, random_state=99)
    dtrain = xgb.DMatrix(x1, y1)
    dvalid = xgb.DMatrix(x2, y2)
    del x1, y1, x2, y2 
    gc.collect()
    watchlist = [(dtrain, 'train'), (dvalid, 'valid')]
    model = xgb.train(params, dtrain, 200, watchlist, maximize=True, early_stopping_rounds = 25, verbose_eval=5)
    del dvalid
else:
    dtrain = xgb.DMatrix(train_df, y)
    del train_df, y
    gc.collect()
    watchlist = [(dtrain, 'train')]
    model = xgb.train(params, dtrain, 30, watchlist, maximize=True, verbose_eval=1)

del dtrain
gc.collect()

print('[{}] Finish XGBoost Training'.format(time.time() - start_time))

# Plot the feature importance from xgboost
plot_importance(model)
plt.gcf().savefig('feature_importance_xgb.png')

# Load the test for predict 
test = pd.read_csv("test.csv", usecols=test_columns, dtype=dtypes)
test = pd.merge(test, ip_count, on='ip', how='left', sort=False)
del ip_count
gc.collect()

sub['click_id'] = test['click_id'].astype('int')

test['clicks_by_ip'] = test['clicks_by_ip'].astype('uint16')
test = timeFeatures(test)
test.drop(['click_id', 'ip'], axis=1, inplace=True)
dtest = xgb.DMatrix(test)
del test
gc.collect()

# Save the predictions
sub['is_attributed'] = model.predict(dtest, ntree_limit=model.best_ntree_limit)
sub.to_csv('xgb_sub.csv', float_format='%.8f', index=False)

train_df.columns.values
 ['ip' 'app' 'device' 'os' 'channel' 'click_time' 'attributed_time'
 'is_attributed']
[57.53672504425049] Finished to load data
[91.68102288246155] Start to generate time features
[91.93506002426147] Start XGBoost Training


XGBoostError: b'[19:30:28] src/objective/regression_obj.cc:43: Check failed: info.labels.size() != 0U (0 vs. 0) label set cannot be empty\n\nStack trace returned 6 entries:\n[bt] (0) 0   libxgboost.dylib                    0x0000001a17299aa8 _ZN4dmlc15LogMessageFatalD2Ev + 40\n[bt] (1) 1   libxgboost.dylib                    0x0000001a17302ff9 _ZN7xgboost3obj10RegLossObjINS0_22LogisticClassificationEE11GetGradientERKNSt3__16vectorIfNS4_9allocatorIfEEEERKNS_8MetaInfoEiPNS5_INS_6detail18bst_gpair_internalIfEENS6_ISG_EEEE + 281\n[bt] (2) 2   libxgboost.dylib                    0x0000001a172960f6 _ZN7xgboost11LearnerImpl13UpdateOneIterEiPNS_7DMatrixE + 1014\n[bt] (3) 3   libxgboost.dylib                    0x0000001a172af73f XGBoosterUpdateOneIter + 79\n[bt] (4) 4   libffi.6.dylib                      0x0000000111130884 ffi_call_unix64 + 76\n[bt] (5) 5   ???                                 0x00007ffee05bd330 0x0 + 140732662534960\n'

In [38]:
import gc
import time
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split
import xgboost as xgb
from xgboost import plot_importance
import matplotlib.pyplot as plt

# Change this for validation with 10% from train
is_valid = False

path = './input/'

def timeFeatures(df):
    # Make some new features with click_time column
    df['datetime'] = pd.to_datetime(df['click_time'])
    df['dow']      = df['datetime'].dt.dayofweek
    df["doy"]      = df["datetime"].dt.dayofyear
    #df["dteom"]    = df["datetime"].dt.daysinmonth - df["datetime"].dt.day
    df.drop(['click_time', 'datetime'], axis=1, inplace=True)
    return df

start_time = time.time()

train_columns = ['ip', 'app', 'device', 'os', 'channel', 'click_time', 'is_attributed']
test_columns  = ['ip', 'app', 'device', 'os', 'channel', 'click_time', 'click_id']
dtypes = {
        'ip'            : 'uint32',
        'app'           : 'uint16',
        'device'        : 'uint16',
        'os'            : 'uint16',
        'channel'       : 'uint16',
        'is_attributed' : 'uint8',
        'click_id'      : 'uint32'
        }

# Read the last lines because they are more impacting in training than the starting lines
train = pd.read_csv(path+"train.csv", skiprows=range(1,123903891), nrows=61000000, usecols=train_columns, dtype=dtypes)
test = pd.read_csv(path+"test_supplement.csv", usecols=test_columns, dtype=dtypes)

print('[{}] Finished to load data'.format(time.time() - start_time))

# Drop the IP and the columns from target
y = train['is_attributed']
train.drop(['is_attributed'], axis=1, inplace=True)

# Drop IP and ID from test rows
sub = pd.DataFrame()
#sub['click_id'] = test['click_id'].astype('int')
test.drop(['click_id'], axis=1, inplace=True)
gc.collect()

nrow_train = train.shape[0]
merge = pd.concat([train, test])

del train, test
gc.collect()

# Count the number of clicks by ip
ip_count = merge.groupby(['ip'])['channel'].count().reset_index()
ip_count.columns = ['ip', 'clicks_by_ip']
merge = pd.merge(merge, ip_count, on='ip', how='left', sort=False)
merge['clicks_by_ip'] = merge['clicks_by_ip'].astype('uint16')
merge.drop('ip', axis=1, inplace=True)

train = merge[:nrow_train]
test = merge[nrow_train:]

del test, merge
gc.collect()

print('[{}] Start to generate time features'.format(time.time() - start_time))

train = timeFeatures(train)
gc.collect()

print('[{}] Start XGBoost Training'.format(time.time() - start_time))

# Set the params(this params from Pranav kernel) for xgboost model
# params = {'eta': 0.3,
#           'tree_method': "hist",
#           'grow_policy': "lossguide",
#           'max_leaves': 1400,  
#           'max_depth': 0, 
#           'subsample': 0.9, 
#           'colsample_bytree': 0.7, 
#           'colsample_bylevel':0.7,
#           'min_child_weight':0,
#           'alpha':4,
#           'objective': 'binary:logistic', 
#           'scale_pos_weight':9,
#           'eval_metric': 'auc', 
#           'nthread':8,
#           'random_state': 99, 
#           'silent': True}
 
params = {'eta': 0.3,
          'tree_method': "hist",
          'grow_policy': "lossguide",
          'max_leaves': 1400,  
          'max_depth': 0, 
          'subsample': 0.9, 
          'colsample_bytree': 0.7, 
          'colsample_bylevel':0.7,
          'min_child_weight':0,
          'alpha':4,
#           'mlogloss': 'multi:softprob',
          'objective': 'binary:logistic', 
          'scale_pos_weight':9,
          'eval_metric': 'auc', 
          'nthread':8,
          'random_state': 99, 
          'silent': True}
 

if (is_valid == True):
    # Get 10% of train dataset to use as validation
    x1, x2, y1, y2 = train_test_split(train, y, test_size=0.1, random_state=99)
    dtrain = xgb.DMatrix(x1, y1)
    dvalid = xgb.DMatrix(x2, y2)
    del x1, y1, x2, y2 
    gc.collect()
    watchlist = [(dtrain, 'train'), (dvalid, 'valid')]
    model = xgb.train(params, dtrain, 200, watchlist, maximize=True, early_stopping_rounds = 25, verbose_eval=5)
    del dvalid
else:
    print('train\n', train.head(10))
    dtrain = xgb.DMatrix(train, y)
    print('dtrain\n', dtrain)
    
    del train, y
    gc.collect()
    watchlist = [(dtrain, 'train')]
#     print('train_before_xgb\n', dtrain)
    model = xgb.train(params, dtrain, 30, watchlist, maximize=True, verbose_eval=1)

del dtrain
gc.collect()

print('[{}] Finish XGBoost Training'.format(time.time() - start_time))

# Plot the feature importance from xgboost
plot_importance(model)
plt.gcf().savefig('feature_importance_xgb.png')

# Load the test for predict 
test = pd.read_csv(path+"test.csv", usecols=test_columns, dtype=dtypes)
test = pd.merge(test, ip_count, on='ip', how='left', sort=False)
del ip_count
gc.collect()

sub['click_id'] = test['click_id'].astype('int')

test['clicks_by_ip'] = test['clicks_by_ip'].astype('uint16')
test = timeFeatures(test)
test.drop(['click_id', 'ip'], axis=1, inplace=True)
dtest = xgb.DMatrix(test)
del test
gc.collect()

# Save the predictions
sub['is_attributed'] = model.predict(dtest, ntree_limit=model.best_ntree_limit)
sub.to_csv('xgb_sub.csv', float_format='%.8f', index=False)

[64.60566782951355] Finished to load data
[105.4098949432373] Start to generate time features
[105.69547700881958] Start XGBoost Training
train
 Empty DataFrame
Columns: [app, device, os, channel, clicks_by_ip, dow, doy]
Index: []
dtrain


XGBoostError: b'[21:08:16] src/objective/regression_obj.cc:43: Check failed: info.labels.size() != 0U (0 vs. 0) label set cannot be empty\n\nStack trace returned 6 entries:\n[bt] (0) 0   libxgboost.dylib                    0x0000001a17299aa8 _ZN4dmlc15LogMessageFatalD2Ev + 40\n[bt] (1) 1   libxgboost.dylib                    0x0000001a17302ff9 _ZN7xgboost3obj10RegLossObjINS0_22LogisticClassificationEE11GetGradientERKNSt3__16vectorIfNS4_9allocatorIfEEEERKNS_8MetaInfoEiPNS5_INS_6detail18bst_gpair_internalIfEENS6_ISG_EEEE + 281\n[bt] (2) 2   libxgboost.dylib                    0x0000001a172960f6 _ZN7xgboost11LearnerImpl13UpdateOneIterEiPNS_7DMatrixE + 1014\n[bt] (3) 3   libxgboost.dylib                    0x0000001a172af73f XGBoosterUpdateOneIter + 79\n[bt] (4) 4   libffi.6.dylib                      0x0000000111130884 ffi_call_unix64 + 76\n[bt] (5) 5   ???                                 0x00007ffee05bd330 0x0 + 140732662534960\n'